In [4]:
from tools.project import INPUT_PATH, LOGS_PATH, OUTPUT_PATH, RAW_PATH
import torch
from audiocraft.data.audio import audio_read, audio_write
from audiocraft.data.audio_utils import convert_audio_channels, convert_audio
import numpy as np
from audioldm_eval.metrics.fad import FrechetAudioDistance
import os
import sys
from fadtk.fad import FrechetAudioDistance, log, calc_frechet_distance
from fadtk.model_loader import CLAPLaionModel, VGGishModel
from fadtk.fad_batch import cache_embedding_files
from audiocraft.models import MusicGen
import shutil
import contextlib
import io
import warnings
import torch.multiprocessing as mp
from toolz import concat
import json
from concurrent.futures import ProcessPoolExecutor
sys.path.append(os.path.abspath("src"))
from src.callbacks import offline_eval
DEVICE = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
EXAMPLES_LEN = 5
torch.cuda.is_available()

True

In [ ]:
model = CLAPLaionModel('music')
# model = VGGishModel()
eval_dir = INPUT_PATH('concepts-dataset', 'data', 'train', '8bit', 'audio')
# cache_embedding_files('fma_pop', model)
# cache_embedding_files(eval_dir, model)
fad = FrechetAudioDistance(model, audio_load_worker=8, load_model=True)
# fad.score('fma_pop', eval_dir)


In [5]:

from toolz import partition_all

def calc_eval(base_dir: str, concepts: list[str], descriptions: dict[str, str], workers=2):
    concepts_batches = list(partition_all(len(concepts) // workers, concepts))
    mp.set_start_method('spawn', force=True)
    with torch.multiprocessing.Pool(processes=workers) as executor:
        results = list(
            executor.starmap(
                offline_eval,
                [
                    (base_dir, batch, {k: descriptions[k] for k in batch})
                    for batch in concepts_batches
                ]
                # [base_dir] * len(concepts_batches),
                # concepts_batches,
                # [{k: descriptions[k] for k in batch} for batch in concepts_batches],
            )
        )

    return {k:v for val in results for k,v in val.items()}
with open(INPUT_PATH('concepts-dataset', "metadata_concepts.json"), "r") as fh:
    concept_descriptions = json.load(fh)
calc_eval('concepts-dataset', list(concept_descriptions.keys()), concept_descriptions, workers=4)
# offline_eval('concepts-dataset', list(concept_descriptions.keys()), concept_descriptions)
# offline_eval('textual-inversion-v3', ['ichika', 'caravan', 'metal', 'ajfa'], {'ichika': 'aa', 'caravan': 'bb', 'metal': 'aa', 'ajfa': 'bb'})

[19:46:05] INFO     Loading HTSAT-base model config.              ]8;id=207;file:///home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/site-packages/laion_clap/clap_module/factory.py\factory.py]8;;\:]8;id=108500;file:///home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/site-packages/laion_clap/clap_module/factory.py#121\121]8;;\
[19:46:05] INFO     Loading HTSAT-base model config.              ]8;id=301205;file:///home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/site-packages/laion_clap/clap_module/factory.py\factory.py]8;;\:]8;id=721358;file:///home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/site-packages/laion_clap/clap_module/factory.py#121\121]8;;\
[19:46:05] INFO     Loading HTSAT-base model config.              ]8;id=70333;file:///home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/site-packages/laion

Loading audio files...: 100%|██████████| 10/10 [00:00<00:00, 173318.35it/s]


[19:46:42] INFO     Loading 10 audio files from                       ]8;id=170459;file:///home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/site-packages/fadtk/fad.py\fad.py]8;;\:]8;id=73048;file:///home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/site-packages/fadtk/fad.py#213\213]8;;\
                    /home/mszawerda/musical-generative-models-conditi           
                    oning/data/output/concepts-dataset/two-steps/temp           
                    ...                                                         


Loading audio files...: 100%|██████████| 10/10 [00:00<00:00, 189787.51it/s]


/home/mszawerda/musical-generative-models-conditioning/data/input/concepts-dataset/data/train/80s-synth/audio.npz
           INFO     Embedding statistics is already cached for        ]8;id=870554;file:///home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/site-packages/fadtk/fad.py\fad.py]8;;\:]8;id=568526;file:///home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/site-packages/fadtk/fad.py#268\268]8;;\
                    /home/mszawerda/musical-generative-models-conditi           
                    oning/data/input/concepts-dataset/data/train/80s-           
                    synth/audio, loading...                                     
/home/mszawerda/musical-generative-models-conditioning/data/output/concepts-dataset/80s-synth/temp.npz
           INFO     Embedding statistics is already cached for        ]8;id=488668;file:///home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/

...

/home/mszawerda/musical-generative-models-conditioning/data/input/concepts-dataset/data/train/pirates/audio.npz
[19:47:30] INFO     Embedding statistics is already cached for        ]8;id=329826;file:///home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/site-packages/fadtk/fad.py\fad.py]8;;\:]8;id=416919;file:///home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/site-packages/fadtk/fad.py#268\268]8;;\
                    /home/mszawerda/musical-generative-models-conditi           
                    oning/data/input/concepts-dataset/data/train/pira           
                    tes/audio, loading...                                       
/home/mszawerda/musical-generative-models-conditioning/data/output/concepts-dataset/pirates/temp.npz
           INFO     Loading embedding files from                      ]8;id=541413;file:///home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/site

Loading audio files...: 100%|██████████| 10/10 [00:00<00:00, 174762.67it/s]


[19:47:41] INFO     Loading 10 audio files from                       ]8;id=535804;file:///home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/site-packages/fadtk/fad.py\fad.py]8;;\:]8;id=148470;file:///home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/site-packages/fadtk/fad.py#213\213]8;;\
                    /home/mszawerda/musical-generative-models-conditi           
                    oning/data/output/concepts-dataset/metal-solos/te           
                    mp...                                                       
[19:47:41] INFO     Loading 10 audio files from                       ]8;id=632708;file:///home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/site-packages/fadtk/fad.py\fad.py]8;;\:]8;id=616075;file:///home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/site-packages/fadtk/fad.py#213\213]8;;\
                    /home/m

Loading audio files...: 100%|██████████| 10/10 [00:00<00:00, 21994.25it/s]


RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 51, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
  File 
"/home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/site-packages/torch/utils/_con
textlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/home/mszawerda/musical-generative-models-conditioning/src/audiocraft/src/callbacks.py", line 106, in 
offline_eval
    score = fad.score(ref_path, gen_path)
  File 
"/home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/site-packages/fadtk/fad.py", 
line 297, in score
    mu_eval, cov_eval = self.load_stats(eval)
  File 
"/home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/site-packages/fadtk/fad.py", 
line 279, in load_stats
    mu, cov = calculate_embd_statistics_online(list(emb_dir.glob("*.npy")))
  File 
"/home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/site-packages/fadtk/utils.py",
line 35, in calculate_embd_statistics_online
    results = pmap(_process_file, files, desc='Calculating statistics')
  File 
"/home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/site-packages/hypy_utils/tqdm_
utils.py", line 21, in pmap
    return process_map(fn, lst, *args, **tqdm_args)
  File 
"/home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/site-packages/tqdm/contrib/con
current.py", line 105, in process_map
    return _executor_map(ProcessPoolExecutor, fn, *iterables, **tqdm_kwargs)
  File 
"/home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/site-packages/tqdm/contrib/con
current.py", line 51, in _executor_map
    return list(tqdm_class(ex.map(fn, *iterables, chunksize=chunksize), **kwargs))
  File "/usr/lib/python3.10/concurrent/futures/process.py", line 766, in map
    results = super().map(partial(_process_chunk, fn),
  File "/usr/lib/python3.10/concurrent/futures/_base.py", line 610, in map
    fs = [self.submit(fn, *args) for args in zip(*iterables)]
  File "/usr/lib/python3.10/concurrent/futures/_base.py", line 610, in <listcomp>
    fs = [self.submit(fn, *args) for args in zip(*iterables)]
  File "/usr/lib/python3.10/concurrent/futures/process.py", line 737, in submit
    self._adjust_process_count()
  File "/usr/lib/python3.10/concurrent/futures/process.py", line 697, in _adjust_process_count
    self._spawn_process()
  File "/usr/lib/python3.10/concurrent/futures/process.py", line 714, in _spawn_process
    p.start()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 118, in start
    assert not _current_process._config.get('daemon'), \
AssertionError: daemonic processes are not allowed to have children
"""

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:23                                                                                   │
│                                                                                                  │
│   20 │   return {k:v for val in results for k,v in val.items()}                                  │
│   21 with open(INPUT_PATH('concepts-dataset', "metadata_concepts.json"), "r") as fh:             │
│   22 │   concept_descriptions = json.load(fh)                                                    │
│ ❱ 23 calc_eval('concepts-dataset', list(concept_descriptions.keys()), concept_descriptions, w    │
│   24 # offline_eval('concepts-dataset', list(concept_descriptions.keys()), concept_descriptio    │
│   25 # offline_eval('textual-inversion-v3', ['ichika', 'caravan', 'metal', 'ajfa'], {'ichika'    │
│   26                                                  

In [ ]:
mu_bg, cov_bg = fad.load_stats('fma_pop')
mu_eval, cov_eval = fad.load_stats(eval_dir)

calc_frechet_distance(mu_bg, cov_bg, mu_eval, cov_eval)

In [ ]:
audio_embeds = fad.load_embeddings(eval_dir)
text_embeds = model.model.get_text_embedding("Guitar backing track in the rock genre, played in B minor, often used for improvisation and jamming.").reshape(-1)
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
np.mean(cosine_similarity(audio_embeds, text_embeds))

In [ ]:
music_model = MusicGen.get_pretrained('facebook/musicgen-small')
music_model.set_generation_params(
	use_sampling=True,
	top_k=250,
	duration=EXAMPLES_LEN
)

In [ ]:
res=music_model.generate([f"music in the style of jazz song"]*5, progress=True)
for a_idx in range(res.shape[0]):
    music = res[a_idx].cpu()
    music = music/np.max(np.abs(music.numpy()))
    path = OUTPUT_PATH("textual-inversion", 'metal', 'temp', f'music_p{a_idx}')
    audio_write(path, music, music_model.cfg.sample_rate)


In [ ]:
cache_embedding_files(OUTPUT_PATH("textual-inversion", 'metal', 'temp'), model)
score = fad.score('fma_pop', OUTPUT_PATH("textual-inversion", 'metal', 'temp'))
shutil.rmtree(os.path.join(OUTPUT_PATH("textual-inversion", 'metal', 'temp'), 'embeddings'))
shutil.rmtree(os.path.join(OUTPUT_PATH("textual-inversion", 'metal', 'temp'), 'convert'))
shutil.rmtree(os.path.join(OUTPUT_PATH("textual-inversion", 'metal', 'temp'), 'stats'))